In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno
import matplotlib.pyplot as plt
from datetime import date
today = date.today()
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn
from wordcloud import WordCloud
from nltk.stem import PorterStemmer
import string
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import pyLDAvis
import pyLDAvis.sklearn
import gensim
from itertools import combinations

## Fonctions utlisées dans le code
* Suppression des stopword
* Lemmatisation : Finalement non utlisée car non appropriée à certains mots
* Supression des tags qui ne sont pas dan le TOP 100
* Traitement du C#
* Fonction vide pour TF_IDF afin de ne pas utiliser les tokenizer natifs

In [ ]:
#Fonction de preprocessing
stop_words = set(stopwords.words('English'))
def removeStopWord(Word_list):
    filtered_Word_list = Word_list[:] #make a copy of the Word_list
    for Word in Word_list: # iterate over Word_list
        if Word.lower() in stop_words: 
            filtered_Word_list.remove(Word) # remove Word from filtered_Word_list if it is a stopword
    return filtered_Word_list

# Instantiate stemmers
porter = PorterStemmer()
#Fonction de lemmatisation
def lemmatisation(Word_list):
    Words = Word_list[:] #make a copy of the Word_list
    Words = [porter.stem(word) for word in Words]
    return Words

#Fonction qui supprime le tag si celui ci n'appartient au TOP        
def removeNotTop100(Word_list):
    filtered_Word_list = Word_list[:] #make a copy of the Word_list
    for Word in Word_list: # iterate over Word_list
        if Word not in Top100: 
            filtered_Word_list.remove(Word) # remove Word from filtered_Word_list if it is a stopword
    if len(filtered_Word_list) == 0:
        return None
    else:
        return filtered_Word_list
    
#Fonction qui supprime le mot si seulement du numérique        
def removeOnlyNumeric(Word_list):
    word_list = Word_list[:] #make a copy of the Word_list
    for Word in Word_list: # iterate over Word_list
        if Word.isnumeric(): 
            word_list.remove(Word) # remove Word from filtered_Word_list if it is a stopword
    return word_list
    
#Fonction qui traite le C#
def processCSharp(Word_list):
    word_list = Word_list[:] #make a copy of the Word_list
    for index, value in enumerate(word_list):
        if value == '#':
            word_list.remove(value) # remove Word from filtered_Word_list if it is a stopword
            word_list[index-1] = 'c#'
    return word_list

# Fonction qui ne fait rien pour exploiter le pré traitement que nous avons réalisé
def dummy(doc):
    return doc

In [ ]:
#Lecture du Fichier
df = pd.read_csv('posts_after_prepocessing.csv', sep = ',', encoding='UTF-8')

In [ ]:
#Préprocessing du champ Tittle et Body
df['TitleBody'] = df['TitleBody'].map(lambda x: BeautifulSoup(x, "html.parser").get_text())
df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace('\n', ' '))
df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace(':', ''))

punct = string.punctuation
for c in punct:
    if c != '#':
        df['TitleBody'] = df['TitleBody'].map(lambda x: x.replace(c, ''))

#Création des nuages de mots avec NLTK
df['TitleBody_final'] = df.apply(lambda row: nltk.word_tokenize(row['TitleBody'],language='english'), axis=1)

#Supression des StopWord
df['TitleBody_final'] = df.apply(lambda row:removeStopWord(row['TitleBody_final']), axis=1)

#Lemmatisation
#df['TitleBody_final'] = df.apply(lambda row:lemmatisation(row['TitleBody_final']), axis=1)

#Traitement du C#
df['TitleBody_final'] = df.apply(lambda row:processCSharp(row['TitleBody_final']), axis=1)

#Suppressin des numériques
df['TitleBody_final'] = df.apply(lambda row:removeOnlyNumeric(row['TitleBody_final']), axis=1)


In [ ]:
df.head(5)

In [ ]:
wordcloud = WordCloud(width = 800, height = 1000,
                max_words=1000,
                background_color ='white',
                min_font_size = 12,
                colormap="Set2").generate(str(df['TitleBody_final']))
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

## Première étape on vectorize avec TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=12000,tokenizer=dummy,
        preprocessor=dummy)
tfidf = tfidf_vectorizer.fit_transform(df["TitleBody_final"])
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

### Selection du nombre de TOPIC
* 'nndsvd': Nonnegative Double Singular Value Decomposition (NNDSVD) initialization (better for sparseness)

In [ ]:
kmin, kmax = 5, 20
topic_models = []
# try each value of k
for k in range(kmin,kmax+1):
    print("Applying NMF for k=%d ..." % k )
    # run NMF
    model = NMF( init="nndsvd", n_components=k,max_iter = 300,random_state=42) 
    W = model.fit_transform(tfidf)
    H = model.components_    
    # store for later
    topic_models.append( (k,W,H) )

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def calculate_coherence(term_rankings ):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append(nlp(pair[0]).similarity(nlp(pair[1])))
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

def get_descriptor( all_terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( all_terms[term_index] )
    return top_terms

In [ ]:
k_values = []
coherences = []
for (k,W,H) in topic_models:
    # Get all of the topic descriptors - the term_rankings, based on top 10 terms
    term_rankings = []
    for topic_index in range(k):
        term_rankings.append( get_descriptor( tfidf_feature_names, H, topic_index, 10 ) )
    k_values.append( k )
    coherences.append( calculate_coherence(term_rankings ) )
    print("K=%02d: Coherence=%.4f" % ( k, coherences[-1] ) )

In [ ]:
fig = plt.figure(figsize=(13,7))
# create the line plot
ax = plt.plot( k_values, coherences )
plt.xticks(k_values)
plt.xlabel("Number of Topics")
plt.ylabel("Mean Coherence")
# add the points
plt.scatter(k_values, coherences, s=120)
# find and annotate the maximum point on the plot
ymax = max(coherences)
xpos = coherences.index(ymax)
best_k = k_values[xpos]
plt.annotate("k=%d" % best_k, xy=(best_k, ymax), xytext=(best_k, ymax), textcoords="offset points", fontsize=16)
# show the plot
plt.show()

## Puis on calcul les topics avec NMF

In [ ]:
#On recupère le nombre de TOPIC optimal, c'est à dire 8
from sklearn.decomposition import NMF
k = 8
# create the model, specifiying the initialization strategy and the number of topics to produce
model = NMF(n_components=k,init="nndsvd",random_state=42,max_iter = 300) 
# apply the model and extract the two factor matrices
W = model.fit_transform(tfidf)
H = model.components_

In [ ]:
def get_descriptor( terms, H, topic_index, top ):
    # reverse sort the values to sort the indices
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    # now get the terms corresponding to the top-ranked indices
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( tfidf_feature_names[term_index] )
    return top_terms

In [ ]:
descriptors = []
for topic_index in range(k):
    descriptors.append( get_descriptor( tfidf_feature_names, H, topic_index, 10 ) )
    str_descriptor = ", ".join( descriptors[topic_index] )
    print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, hamming_loss,jaccard_score
#Fonction qui renvoi différents scores
def print_score(y_test, y_pred):
    print("Hamming loss : {}".format(hamming_loss(y_test, y_pred)))
    print('Subset Accuracy : ', accuracy_score(y_test, y_pred, normalize=True, sample_weight=None))
    print('F1-score : ', f1_score(y_test, y_pred, average='micro'))
    print('Jaccard : ', jaccard_score(y_test, y_pred, average='micro'))
    #Revoir les indicateurs avec Jacard

## Nommage TOPIC
* Exemple de code
* GIT
* Traitement des chaines de caractères
* Language Frontend (C#,HTML, CSS)
* Shell
* Gestion des tableaux
* Base de données (SQL, MySQL)
* Python

In [ ]:
def plot_top_term_weights( tfidf_feature_names, H, topic_index, top ):
    # get the top terms and their weights
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    top_weights = []
    for term_index in top_indices[0:top]:
        top_terms.append( tfidf_feature_names[term_index] )
        top_weights.append( H[topic_index,term_index] )
    # note we reverse the ordering for the plot
    top_terms.reverse()
    top_weights.reverse()
    # create the plot
    fig = plt.figure(figsize=(13,8))
    # add the horizontal bar chart
    ypos = np.arange(top)
    ax = plt.barh(ypos, top_weights, align="center", color="green",tick_label=top_terms)
    plt.xlabel("Term Weight",fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_top_term_weights(tfidf_feature_names, H, 0, 15)

In [ ]:
plot_top_term_weights(tfidf_feature_names, H, 1, 15)

# LDA : Latent Dirichlet Allocation après bag of Words
* Détermination des TOPICS
Quels sont les thèmes, les distributions de chaque mot sur les thèmes, la fréquence d’apparition de chaque thème sur le corpus

In [ ]:
bow_vectorizer = CountVectorizer(ngram_range = (1,1),
                                 tokenizer=dummy,preprocessor=dummy)
docs_bow = bow_vectorizer.fit_transform(df["TitleBody_final"])

In [ ]:
lda = LatentDirichletAllocation(random_state=42,
                                batch_size=400)
params = {'n_components' : [5, 6, 7,8],
          "learning_decay" : [0.7, 0.9],
          "learning_method" : ["batch", "online"]}

#params = {'n_components' : [6,7,8]}

gridsearch_lda = GridSearchCV(lda,
                              param_grid=params,
                              cv=5,
                              verbose=2)
gridsearch_lda.fit(docs_bow)

#1 heure d'exécution. Résultat obtenu
#Best Model's Params:  {'learning_decay': 0.9, 'learning_method': 'online', 'n_components': 8}
#Best Log Likelihood Score:  -1689928.643148844

In [ ]:
best_lda_model = gridsearch_lda.best_estimator_

print("Best Model's Params: ", gridsearch_lda.best_params_)
print("Best Log Likelihood Score: ", gridsearch_lda.best_score_)

In [ ]:
panel = pyLDAvis.sklearn.prepare(best_lda_model, docs_bow, bow_vectorizer, mds='tsne')
pyLDAvis.display(panel)